In [1]:
import pandas as pd
import json
import ast

In [2]:
pd.set_option('display.max_colwidth', None) ##para ver todo el contenido de la columna

In [3]:
# traigo el dataset credits a un dataframe y le cambio el nombre de la columna del id(pelicula) a id_original
dfcredits= pd.read_csv(r'C:\Users\flori\Desktop\DATA SCIENCE\LABS\credits\credits.csv')
dfcredits.rename(columns={'id':'id_original'}, inplace= True)


In [4]:
#utilizo ast literal para evaluar y transformar las cadenas de texto que representan diccionarios a diccionarios propiamente dichos
dfcredits['crew'] = dfcredits['crew'].apply(ast.literal_eval) 

In [5]:
df_exploded = dfcredits.explode('crew')

def extract_crew_info(x):
    if isinstance(x, dict):
        return pd.Series([x.get('credit_id', ''), x.get('department', ''),x.get('gender', ''),x.get('id', ''),x.get('job', ''),x.get('name', ''),x.get('profile_path', '')  ])
    else:
        return pd.Series(['', ''])

df_exploded[['credit_id', 'department', 'gender','id_crew','job', 'name', 'profile_path']] = df_exploded['crew'].apply(extract_crew_info) 

In [6]:
df = dfcredits.join(df_exploded[['credit_id', 'department', 'gender', 'id_crew', 'job', 'name', 'profile_path']]) 

In [7]:
columna_eliminada = ['crew','cast', 'credit_id','profile_path', 'gender']
df.drop(columns=columna_eliminada, inplace=True) 




In [8]:
df.head()

,id_original,department,id_crew,job,name
0,862,Directing,7879.0,Director,John Lasseter
0,862,Writing,12891.0,Screenplay,Joss Whedon
0,862,Writing,7.0,Screenplay,Andrew Stanton
0,862,Writing,12892.0,Screenplay,Joel Cohen
0,862,Writing,12893.0,Screenplay,Alec Sokolow


In [9]:
# Convertir las columnas numéricas a cadenas
df['id_crew'] = df['id_crew'].astype(str)
#df['gender'] = df['gender'].astype(str) la borre
df['id_original'] = df['id_original'].astype(str) 

In [10]:
director_resultado = df[df['job'] == 'Director']['job'].value_counts()
director_resultado 

Director    49048
Name: job, dtype: int64

In [11]:
df_director= df[df['job']== 'Director']
df_director.head()

,id_original,department,id_crew,job,name
0,862,Directing,7879.0,Director,John Lasseter
1,8844,Directing,4945.0,Director,Joe Johnston
2,15602,Directing,26502.0,Director,Howard Deutch
3,31357,Directing,2178.0,Director,Forest Whitaker
4,11862,Directing,56106.0,Director,Charles Shyer


In [12]:
df_director.shape

(49048, 5)

In [13]:
grouped_df = df_director.groupby('id_original').agg({'name': lambda x: "'" + "' '".join(x) + "'",
                                           'id_crew': lambda x: "'" + ','.join(map(str, x)) + "'"}).reset_index()

grouped_df 

,id_original,name,id_crew
0,100,'Guy Ritchie','956.0'
1,10000,'Sergio Cabrera','61710.0'
2,10001,'Yahoo Serious','61797.0'
3,100010,'Frank Borzage','14855.0'
4,100017,'Angelina Maccarone','40016.0'
...,...,...,...
44540,99946,'Sam Taylor','143561.0'
44541,9995,'Robert Adetuyi','31120.0'
44542,9997,'Shane Abbess','61763.0'
44543,99977,'Dom DeLuise','6844.0'


In [14]:
grouped_df.to_csv('directoresPorpeliID.csv', index=False)

In [15]:
dfori= grouped_df[grouped_df['id_original']=='826']
dfori

,id_original,name,id_crew
40999,826,'David Lean','12238.0'


In [16]:
# identificar Director por cada Id Pelicula

id_director = df_director[df_director['id_original'] == '862']
director_names = id_director['name'].values[0].split(',')
director_name = director_names[0]

print(f"ID: 862, Director: {director_name}")


ID: 862, Director: John Lasseter


In [17]:
dfcredits1= dfcredits['crew']